# **Demo: Text Summarizer**

In this demo, you will create an Arxiv paper summarizer that downloads a paper as a PDF, reads it in one shot, and generates a summary. You can use the **The Impact of Generative Artificial Intelligence** paper as an example for demonstration.

## **Steps to Perform:**

*   Step 1: Import the Necessary Libraries
*   Step 2: Download and Read the PDF
*   Step 3: Extract Text from the PDF
*   Step 4: Count the Tokens in the Extracted Text
*   Step 5: Use LangChain to Generate a Summary of the Paper



### **Step 1: Import the Necessary Libraries**

In [ ]:
%pip install PyPDF2
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
import os
import requests
from PyPDF2 import PdfReader
import tiktoken
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback

### **Step 2: Download and Read the PDF**

*   Define the path of the paper.
*   Read the PDF using **PdfReader**.
*   Print the number of pages.



In [ ]:
# Define the path of the paper
PAPER_PATH = "arxiv_impact_of_GENAI.pdf"

# Read the PDF
reader = PdfReader(PAPER_PATH)

# Print the number of pages in the PDF
print(f"Number of pages: {len(reader.pages)}")

Number of pages: 9


### **Step 3: Extract Text from the PDF**

*   Initialize an empty list to store text parts.
*   Define a function to visit the body of the text.
*   Extract the text from each page of the PDF.
*   Join the parts of the text into a single string.
*   Print the extracted text.




In [ ]:
# Initialize an empty list to store the parts of the text
parts = []

# Define a function to visit the body of the text
def visitor_body(text, cm, tm, fontDict, fontSize):
    y = tm[5]
    if y > 50 and y < 720:
        parts.append(text)

# Extract the text from each page of the PDF
for page in reader.pages:
    page.extract_text(visitor_text=visitor_body)

# Join the parts of the text into a single string
text_body = "".join(parts)

# Print the extracted text
print(text_body)


The Impact of Generative Artificial Intelligence
Kaichen Zhang1,2, Ohchan Kwon2, Hui Xiong1,2
1Hong Kong University of Science and Technology (Guangzhou)
2Hong Kong University of Science and Technology
kzhangbi@connect.ust.hk, ohchankw@ust.hk, xionghui@ust.hk
Abstract
The rise of generative artificial intelligence (AI) has sparked
concerns about its potential influence on unemployment and
market depression. This study addresses this concern by ex-
amining the impact of generative AI on product markets. To
overcome the challenge of causal inference, given the inher-
ent limitations of conducting controlled experiments, this pa-
per identifies an unanticipated and sudden leak of a highly
proficient image-generative AI as a novel instance of a ”natu-
ral experiment.” This AI leak spread rapidly, significantly re-
ducing the cost of generating anime-style images compared to
other styles, creating an opportunity for comparative assess-
ment. We collect real-world data from an artwork outsou

### **Step 4: Count the Tokens in the Extracted Text**

*   Define a function to count the tokens in a text string.
*   Count the tokens in the extracted text.
*   Print the number of tokens.



In [ ]:
# Define a function to count the tokens in a text string
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Count the tokens in the extracted text
num_tokens = num_tokens_from_string(text_body, "gpt-3.5-turbo")

# Print the number of tokens
print(num_tokens)

10556


### **Step 5: Use LangChain to Generate a Summary of the Paper**

*   Define the system and human prompts.
*   Create the **ChatPromptTemplate** object.
*   Create the **ChatOpenAI** object.
*   Create the **LLMChain** object.
*   Run the **LLMChain**.
*   Print the output.




In [ ]:
import openai
from google.colab import userdata

api_key = userdata.get('secret_key')

openai.api_key = api_key

In [ ]:
# Define the system prompt to set model's behavior
context_template = "You are a helpful AI Researcher that specializes in analyzing ML, AI, and LLM papers. Please use all your expertise to approach this task. Output your content in markdown format and include titles where relevant."
system_message_prompt = SystemMessagePromptTemplate.from_template(context_template)

# Define the human prompt (task)
human_template = "Please summarize this paper focusing on the key important takeaways for each section. Expand the summary on methods so they can be clearly understood. \n\n PAPER: \n\n{paper_content}"
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=human_template,
            input_variables=["paper_content"],
        )
    )

# Create the ChatPromptTemplate object
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Create the ChatOpenAI object
chat = ChatOpenAI(openai_api_key=api_key, model_name="gpt-3.5-turbo-16k", temperature=0.2)

# Create the LLMChain object
summary_chain = LLMChain(llm=chat, prompt=chat_prompt_template)

# Run the LLMChain and print the output
with get_openai_callback() as cb:
    output = summary_chain.run(text_body)
print(output)

/tmp/ipython-input-8-149472759.py:18: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(openai_api_key=api_key, model_name="gpt-3.5-turbo-16k", temperature=0.2)
/tmp/ipython-input-8-149472759.py:21: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summary_chain = LLMChain(llm=chat, prompt=chat_prompt_template)
/tmp/ipython-input-8-149472759.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = summary_chain.run(text_body)


# Summary of "The Impact of Generative Artificial Intelligence"

## Abstract
The paper explores the impact of generative AI on product markets using a natural experiment of a leaked image-generative AI. Surprisingly, the results show that generative AI lowers average prices but significantly boosts order volume and overall revenue, benefiting artists and fostering market prosperity.

## Introduction
Generative AI has raised concerns about workforce displacement and market depression. The paper addresses these concerns by empirically examining the impact of generative AI on product markets using a unique natural experiment.

## Methods
- **Causal Inference Challenge**: The paper overcomes the challenge of causal inference by leveraging an unexpected leak of a highly proficient image-generative AI as a natural experiment.
- **Data Collection**: Real-world data from a major Chinese painting outsourcing platform was collected from January 2022 to March 2023.
- **Empirical Strategy**: Diffe

### **Conclusion**

This demo provides a step-by-step guide on how to build an Arxiv paper summarizer using Python. By following these steps, you have created your own paper summarizer that can read a paper in one shot and generate a summary.

**Note:**
*   Save the output as a text file.
*   Name the text file **Summary.txt**.
*   This file will be used in the next session for benchmarking purposes.



In [ ]:
# Save the output to a text file
with open("Summary.txt", "w") as file:
    file.write(output)